In [1]:
import numpy as np
import pandas as pd
import math
from datetime import time

import CMAPSAuxFunctions
from data_handler_VALVE import ValveDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

/home/cmartin28/anaconda3/envs/work/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cmartin28/anaconda3/envs/work/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


<h1> Create Data Handler </h1>

In [2]:
features = ['timestamp', 'externalControllerOutput', 'undisturbedMediumFlow', 'pressureValveInlet', 'pressureValveOutlet', 'mediumTemperature', 'rodDisplacement', 'disturbedMediumFlow', 'selectedFault', 'faultType', 'faultIntensity']
selected_indices = np.array([2,4,5,6,7,8])
selected_features = list(features[i] for i in selected_indices-1)

window_size = 30
window_stride = 1

# min = 2018-02-14 18:59:20
# max = 2018-08-19 18:28:20
time_start = "2018-02-14 18:59:20"
time_end = "2018-08-19 18:28:20"

# Either classification or regression
problem = 'classification'
#problem = 'regression'

vHandler = ValveDataHandler(time_start, time_end, selected_features = selected_features,
                            sequence_length = window_size, sequence_stride = window_stride,
                            problem = problem)

init


<h1>Keras Model </h1>

In [3]:
# np.set_printoptions(threshold=np.nan)

K.clear_session()
lambda_regularization = 0.20

def create_ANN_model(input_shape, problem):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim = input_shape, activation = 'relu', kernel_initializer = 'glorot_normal', 
                    kernel_regularizer = regularizers.l2(lambda_regularization), name = 'fc1'))
    
    if (problem == 'classification'):
        model.add(Dense(2, activation = 'softmax', name = 'out'))
    elif (problem == 'regression'):
        model.add(Dense(1, activation = 'linear', name = 'out'))
        
    #model.add(Dense(1, activation = 'softmax', name = 'out'))
    
    return model

<h1>Tunable Model </h1>

In [4]:
scaler = MinMaxScaler(feature_range = (-1, 1))
#scaler = StandardScaler()

In [5]:
def get_compiled_model(shape, problem):
    
    K.clear_session()
    
    if (problem == 'classification'):
        # Parameters for the model
        optimizer = SGD(lr = 0.01, momentum = 0.0, decay = 0.0, nesterov = False)
        loss_function = 'categorical_crossentropy'
        metrics = ['accuracy']
    elif (problem == 'regression'):
        # Parameters for the model
        optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
        loss_function = 'mean_squared_error'
        metrics = ['mse']
        
    model = None
    
    # Create and compile the model
    model = create_ANN_model(shape, problem)
    model.compile(optimizer = optimizer, loss = loss_function, metrics = metrics)
    
    return model

In [6]:
num_features = len(selected_features)
input_shape = num_features * window_size

model = get_compiled_model(input_shape, problem)
tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

tModel.data_handler.data_scaler = None
tModel.data_scaler = scaler

<h1> Loading Data from MySQL Database </h1>

In [7]:
#vHandler.connect_to_db("remoteAdmin","remoteAdmin","169.236.181.40:3306","damadics")

In [8]:
# Extract data from database
# vHandler.extract_data_from_db()
# vHandler.extract_data_from_db()

In [9]:
# Extract data from .csv file. If loading from .csv, don't forget to uncomment .extract_data_from_db() in data_handler_VALVE'''
vHandler._df = pd.read_csv('valve_dataset.csv', sep = ',')
# adds a new feature called "Status" on whether the valve is broken '1' or not broken '0'
#vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)
# this adds a column to vHandler._X
#vHandler._X = vHandler._df[selected_features]
#vHandler._X.insert(loc = 0, column = 'id', value = np.arange(len(vHandler._df)))
# vHandler._X = vHandler._X.values

# vHandlezr._y = vHandler._df['status'].values

In [10]:
print(vHandler._df)

                  timestamp  externalControllerOutput  undisturbedMediumFlow  \
1       2018-02-14 18:59:20                  0.500000                    NaN   
2       2018-02-14 19:00:20                  0.366043                    NaN   
3       2018-02-14 19:01:20                  0.732444                    NaN   
4       2018-02-14 19:02:20                  0.257854                    NaN   
5       2018-02-14 19:03:20                  0.659356                    NaN   
6       2018-02-14 19:04:20                  0.484302                    NaN   
7       2018-02-14 19:05:20                  0.366043                    NaN   
8       2018-02-14 19:06:20                  0.732444                    NaN   
9       2018-02-14 19:07:20                  0.257854                    NaN   
10      2018-02-14 19:08:20                  0.659356                    NaN   
11      2018-02-14 19:09:20                  0.484302                    NaN   
12      2018-02-14 19:10:20             

In [11]:
vHandler._X = vHandler._df[selected_features].values

if (problem == 'classification'):
    vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)
    vHandler._y = vHandler._df["status"].values
elif (problem == 'regression'):
    vHandler._y = vHandler._df["selectedFault"].values
        
# vHandler._X = vHandler._df[selected_features].values
# vHandler._y = vHandler._df["selectedFault"].values
#vHandler._y = list(map(lambda valve: 0 if valve == 20 else 1, vHandler._y))
#vHandler._y = vHandler._df["status"].values

In [12]:
print(vHandler._X)
print(vHandler._y)

[[5.00000e-01 8.76840e-01 6.50832e-01 2.14553e-01 1.59000e-03 1.00000e+00]
 [3.66043e-01 8.50576e-01 6.43717e-01 2.15373e-01 3.69939e-01 1.00000e+00]
 [7.32444e-01 8.49537e-01 6.47472e-01 2.14781e-01 7.32446e-01 2.22181e-01]
 ...
 [7.32444e-01 8.47251e-01 6.43362e-01 2.14299e-01 1.00000e+00 0.00000e+00]
 [2.57854e-01 8.49342e-01 6.43664e-01 2.15743e-01 1.00000e+00 3.34000e-04]
 [6.59356e-01 8.49701e-01 6.43186e-01 2.17503e-01 1.00000e+00 0.00000e+00]]
[0 0 0 ... 1 1 1]


In [13]:
#vHandler.load_data(cross_validation_ratio = 0.3, test_ratio = 0.2, unroll = True)
tModel.load_data(unroll = True, verbose = 0, cross_validation_ratio = 0.2, test_ratio = 0.2)

Loading data from database
One-hot-encoding: 0:00:00.199354
265100
Splitting into samples: 0:00:01.201014
counter: 129
265100
Splitting into samples: 0:00:01.103873
counter: 129
Train, cv, and test splitting: 0:00:00.000276
Sequence length 30
Sequence stride 1
X_train len 79
X_crossVal len 25
X_test len 25
y_train len 79
y_crossVal len 25
y_test len 25
X_train[0]
(244, 6)
[[3.66043e-01 8.49179e-01 6.52448e-01 2.16609e-01 1.00000e+00 2.65400e-03]
 [7.32444e-01 8.48544e-01 6.56325e-01 2.15544e-01 1.00000e+00 0.00000e+00]
 [2.57854e-01 8.50103e-01 6.57101e-01 2.14507e-01 1.00000e+00 7.17000e-04]
 ...
 [4.84302e-01 8.49889e-01 6.53316e-01 2.15091e-01 9.99008e-01 1.60700e-03]
 [3.66043e-01 8.49532e-01 6.56590e-01 2.15337e-01 1.00000e+00 1.25000e-03]
 [7.32444e-01 8.49245e-01 6.54704e-01 2.14850e-01 9.99328e-01 2.39000e-04]]
79
X_crossVal[-1]
(577, 6)
[[3.66043e-01 8.50569e-01 6.44994e-01 2.15286e-01 9.97533e-01 2.33000e-04]
 [7.32444e-01 8.49335e-01 6.42191e-01 2.14522e-01 9.98811e-01 1.017

In [14]:
vHandler.print_data(True)

#tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

Printing shapes

Training data (X, y)
(162730, 180)
(162730, 2)
Cross-Validation data (X, y)
(25, 180)
(25, 2)
Testing data (X, y)
(25, 180)
(25, 2)
Printing first 5 elements

Training data (X, y)
[[3.66043e-01 8.49179e-01 6.52448e-01 2.16609e-01 1.00000e+00 2.65400e-03
  7.32444e-01 8.48544e-01 6.56325e-01 2.15544e-01 1.00000e+00 0.00000e+00
  2.57854e-01 8.50103e-01 6.57101e-01 2.14507e-01 1.00000e+00 7.17000e-04
  6.59356e-01 8.47265e-01 6.54476e-01 2.16285e-01 1.00000e+00 0.00000e+00
  4.84302e-01 8.49534e-01 6.51210e-01 2.15467e-01 1.00000e+00 0.00000e+00
  3.66043e-01 8.50313e-01 6.48494e-01 2.15205e-01 9.98455e-01 0.00000e+00
  7.32444e-01 8.50344e-01 6.45128e-01 2.16038e-01 1.00000e+00 0.00000e+00
  2.57854e-01 8.49133e-01 6.43467e-01 2.15108e-01 1.00000e+00 0.00000e+00
  6.59356e-01 8.46750e-01 6.46116e-01 2.17283e-01 1.00000e+00 1.67000e-04
  4.84302e-01 8.52029e-01 6.47861e-01 2.14542e-01 9.99346e-01 5.19000e-04
  3.66043e-01 8.50317e-01 6.51685e-01 2.15082e-01 1.00000e+00 0

<h1> Train model and test Tunable Model functionalities </h1>

In [15]:
print(len(vHandler._X_train))
print(len(vHandler._y_train))
print(len(tModel._data_handler._X_train))
print(len(tModel._data_handler.y_train))
print(tModel._data_handler._X_train)
print(tModel._data_handler._y_train)

162730
162730
162730
162730
[[3.66043e-01 8.49179e-01 6.52448e-01 ... 2.15558e-01 9.99633e-01
  1.66100e-03]
 [7.32444e-01 8.48544e-01 6.56325e-01 ... 2.15717e-01 1.00000e+00
  0.00000e+00]
 [2.57854e-01 8.50103e-01 6.57101e-01 ... 2.15708e-01 1.00000e+00
  2.23700e-03]
 ...
 [6.59356e-01 8.50878e-01 6.43726e-01 ... 2.11633e-01 1.00000e+00
  1.37800e-03]
 [4.84302e-01 8.49230e-01 6.47702e-01 ... 2.14322e-01 9.97730e-01
  1.35000e-04]
 [3.66043e-01 8.49123e-01 6.54073e-01 ... 2.17384e-01 9.96252e-01
  0.00000e+00]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [16]:
num_features = len(selected_features)

#lrate = 0.001
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

tModel.epochs = 100
tModel.train_model(learningRate_scheduler = lrate, verbose = 1)

Epoch 1/100
162730/162730 [==============================] - 3s 17us/step - loss: 7.1130 - acc: 0.6922
Epoch 2/100
162730/162730 [==============================] - 2s 12us/step - loss: 5.6399 - acc: 0.7080
Epoch 3/100
162730/162730 [==============================] - 2s 11us/step - loss: 4.5039 - acc: 0.7118
Epoch 4/100
162730/162730 [==============================] - 2s 14us/step - loss: 3.6250 - acc: 0.7124
Epoch 5/100
162730/162730 [==============================] - 3s 16us/step - loss: 2.9445 - acc: 0.7125
Epoch 6/100
162730/162730 [==============================] - 2s 12us/step - loss: 2.4176 - acc: 0.7125
Epoch 7/100
162730/162730 [==============================] - 2s 14us/step - loss: 2.0093 - acc: 0.7126
Epoch 8/100
162730/162730 [==============================] - 1s 8us/step - loss: 1.6929 - acc: 0.7122
Epoch 9/100
162730/162730 [==============================] - 1s 8us/step - loss: 1.4477 - acc: 0.7118
Epoch 10/100
162730/162730 [==============================] - 1s 8us/step -

162730/162730 [==============================] - 1s 6us/step - loss: 0.5987 - acc: 0.7155
Epoch 81/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5987 - acc: 0.7155
Epoch 82/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5987 - acc: 0.7155
Epoch 83/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5986 - acc: 0.7154
Epoch 84/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5986 - acc: 0.7156
Epoch 85/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5986 - acc: 0.7155
Epoch 86/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5985 - acc: 0.7155
Epoch 87/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5985 - acc: 0.7156
Epoch 88/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.5985 - acc: 0.7156
Epoch 89/100
162730/162730 [==============================] - 1s 6us/step - loss: 0.59